<a href="https://colab.research.google.com/github/sggyuan/GNN/blob/main/%E8%89%AF%E5%A5%BD%E8%A1%A8%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.4.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 29.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = torch.load('/content/drive/MyDrive/my_graph_data-bal.pt')

<ipython-input-4-e9945f3ad513>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('/content/drive/MyDrive/my_graph_data-bal.pt')


In [9]:
data = train_test_split_edges(data)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [10]:
data

Data(x=[16283, 13], y=[100000], val_pos_edge_index=[2, 5000], val_pos_edge_attr=[5000, 8], test_pos_edge_index=[2, 10000], test_pos_edge_attr=[10000, 8], train_pos_edge_index=[2, 98972], train_pos_edge_attr=[98972, 8], train_neg_adj_mask=[16283, 16283], val_neg_edge_index=[2, 5000], test_neg_edge_index=[2, 10000])

In [39]:
class GCNEncoder(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=64):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, out_channels)

        self.bn1 = nn.BatchNorm1d(hidden_channels)
        self.bn2 = nn.BatchNorm1d(hidden_channels)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x, edge_index):
        # First GCN layer
        x1 = F.leaky_relu(self.bn1(self.conv1(x, edge_index)))
        x1 = self.dropout(x1)

        # Second GCN layer with residual connection
        x2 = F.leaky_relu(self.bn2(self.conv2(x1, edge_index))) + x1
        x2 = self.dropout(x2)

        # Final GCN layer
        x3 = self.conv3(x2, edge_index)

        return x3


In [11]:
from torch_geometric.nn import GAE

In [16]:
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [48]:
out_channels = 2
num_features = data.num_features
epochs = 100

model = GAE(GCNEncoder(num_features, out_channels))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=10, factor=0.5)

In [41]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [42]:
import torch.nn.functional as F

In [49]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 001, AUC: 0.7703, AP: 0.6945
Epoch: 002, AUC: 0.8512, AP: 0.8044
Epoch: 003, AUC: 0.8808, AP: 0.8590
Epoch: 004, AUC: 0.8960, AP: 0.8922
Epoch: 005, AUC: 0.9022, AP: 0.9080
Epoch: 006, AUC: 0.9048, AP: 0.9158
Epoch: 007, AUC: 0.9058, AP: 0.9195
Epoch: 008, AUC: 0.9059, AP: 0.9222
Epoch: 009, AUC: 0.9054, AP: 0.9228
Epoch: 010, AUC: 0.9046, AP: 0.9230
Epoch: 011, AUC: 0.9035, AP: 0.9227
Epoch: 012, AUC: 0.9022, AP: 0.9220
Epoch: 013, AUC: 0.9012, AP: 0.9212
Epoch: 014, AUC: 0.9006, AP: 0.9201
Epoch: 015, AUC: 0.9002, AP: 0.9193
Epoch: 016, AUC: 0.8966, AP: 0.9164
Epoch: 017, AUC: 0.8894, AP: 0.9111
Epoch: 018, AUC: 0.8742, AP: 0.9010
Epoch: 019, AUC: 0.8523, AP: 0.8839
Epoch: 020, AUC: 0.8216, AP: 0.8534
Epoch: 021, AUC: 0.7608, AP: 0.7717
Epoch: 022, AUC: 0.3971, AP: 0.4545
Epoch: 023, AUC: 0.1318, AP: 0.3292
Epoch: 024, AUC: 0.0944, AP: 0.3192
Epoch: 025, AUC: 0.0836, AP: 0.3167
Epoch: 026, AUC: 0.0769, AP: 0.3153
Epoch: 027, AUC: 0.0713, AP: 0.3142
Epoch: 028, AUC: 0.0683, AP: